In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.linear_model import LogisticRegression

final_train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data_train_downsampled.csv", sep = ',')
final_train.head(5)


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,promotion_flag,srch_destination_id,srch_length_of_stay,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,histpricediff,histstardiff,pricediff,stardiff,reviewscorediff,locationscore1diff,locationscore2diff,pricechange,hotel_count,popularity_score,is219,usd_diff,prop_starrating_monotonic,room_count_booking_window,people_stay_count,month,target
0,1,12,187,3.45,152.24,219,50984,0.0,0,1.61,0.0000,4.14,35,0,23246,1,1,0.0,386.60,1,0.0,0.0,0.0,66.87,1.45,-78.348929,-1.071429,-3.482143,-0.689643,-0.045493,0.555576,104,104.000000,0,66.87,1.31206,0,4,4,0
1,1,12,187,3.45,152.24,219,68914,3.0,1,2.20,0.0206,4.44,13,0,23246,1,1,0.0,386.60,1,0.0,0.0,0.0,51.35,1.45,-62.828929,-1.071429,-0.482143,-0.099643,-0.024893,0.625685,584,496.400000,0,51.35,1.31206,0,4,4,2
2,4,5,219,3.45,152.24,219,58696,4.5,0,1.61,0.0823,5.39,29,0,12588,2,1,0.0,236.91,1,0.0,0.0,0.0,-66.76,3.45,-46.562500,-3.093750,0.593750,-0.894063,-0.081466,0.000000,168,56.000000,1,66.76,3.31206,46,1,12,0
3,4,5,219,3.45,152.24,219,49152,4.5,1,3.09,0.1417,5.92,9,0,12588,2,1,0.0,238.30,1,0.0,0.0,0.0,-176.76,-0.55,63.437500,0.906250,0.593750,0.585937,-0.022066,0.100703,507,295.750000,1,176.76,0.68794,46,1,12,0
4,4,5,219,3.45,152.24,219,139893,3.0,1,1.61,0.0309,5.25,16,0,12588,2,1,0.0,236.14,1,0.0,0.0,0.0,23.24,1.45,-136.562500,-1.093750,-0.906250,-0.894063,-0.132866,0.040323,358,208.833333,1,23.24,1.31206,46,1,12,0


In [0]:
#Selected Features:
#'visitor_location_country_id', 'prop_country_id', 'prop_id', 'srch_destination_id', 'prop_location_score1', 'prop_location_score2', 'visitor_hist_starrating', 'visitor_hist_adr_usd',
selected_features = ['prop_brand_bool', 'prop_log_historical_price', 'promotion_flag',
                     'srch_length_of_stay', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 
                     'histpricediff', 'histstardiff', 'pricediff', 'stardiff', 'reviewscorediff', 'locationscore1diff', 'locationscore2diff', 'pricechange', 'hotel_count', 'popularity_score', 'is219', 
                     'usd_diff', 'prop_starrating_monotonic', 'room_count_booking_window', 'people_stay_count', 'month']

target_features = ['target']

In [0]:
final_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443758 entries, 0 to 443757
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   srch_id                      443758 non-null  int64  
 1   site_id                      443758 non-null  int64  
 2   visitor_location_country_id  443758 non-null  int64  
 3   visitor_hist_starrating      443758 non-null  float64
 4   visitor_hist_adr_usd         443758 non-null  float64
 5   prop_country_id              443758 non-null  int64  
 6   prop_id                      443758 non-null  int64  
 7   prop_review_score            443758 non-null  float64
 8   prop_brand_bool              443758 non-null  int64  
 9   prop_location_score1         443758 non-null  float64
 10  prop_location_score2         443758 non-null  float64
 11  prop_log_historical_price    443758 non-null  float64
 12  position                     443758 non-null  int64  
 13 

In [0]:
#np.isnan(final_train).any()

In [0]:
#np.all(np.isfinite(final_train))

In [0]:
#final_train['pricechange'] = final_train['pricechange'].apply(lambda x: 0 if (x == np.inf) else x)
# final_train.index[np.isinf(X).any(1)]

In [0]:
final_train.replace([np.inf, -np.inf], np.nan, inplace=True)
final_train.fillna(0, inplace=True)

In [0]:
# final_train.index[np.isinf(X).any(1)]

In [0]:
final_train.iloc[214695]

srch_id                        161118.000000
site_id                            18.000000
visitor_location_country_id       164.000000
visitor_hist_starrating             3.450000
visitor_hist_adr_usd              152.240000
prop_country_id                   164.000000
prop_id                         52954.000000
prop_review_score                   4.500000
prop_brand_bool                     0.000000
prop_location_score1                3.740000
prop_location_score2                0.079800
prop_log_historical_price           3.870000
position                            2.000000
promotion_flag                      1.000000
srch_destination_id             17232.000000
srch_length_of_stay                 3.000000
srch_saturday_night_bool            0.000000
srch_query_affinity_score           0.000000
orig_destination_distance         386.600000
random_bool                         0.000000
comp1_rate                          0.000000
comp1_inv                           0.000000
comp1_rate

In [0]:
# create X (features) and y (response)
X = final_train[selected_features]
y = final_train['target']

In [0]:
#np.any(np.isnan(X))

In [0]:
#np.all(np.isfinite(X))

In [0]:
#X = X.reset_index()

In [0]:
#np.any(np.isnan(X))

In [0]:
#np.all(np.isfinite(X))

In [0]:
#np.any(np.isnan(y))

In [0]:
#np.all(np.isfinite(y))

In [0]:
#np.isfinite(X)

In [0]:
#X.replace([np.inf, -np.inf], np.nan)

In [0]:
# # use train/test split with different random_state values
# # we can change the random_state values that changes the accuracy scores
# # the scores change a lot, this is why testing scores is a high-variance estimate
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # check classification scores of logistic regression
# logreg = LogisticRegression()
# logreg.fit(X_train, y_train)
# y_pred = logreg.predict(X_test)
# y_pred_proba = logreg.predict_proba(X_test)[:, 1]
# [fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
# print('Train/Test split results:')
# print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
# print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
# print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))


In [0]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [0]:


# 10-fold cross-validation logistic regression
logreg = LogisticRegression()
logreg.max_iter = 7600
# Use cross_val_score function
# We are passing the entirety of X and y, not X_train or y_train, it takes care of splitting the data
# cv=10 for 10 folds
# scoring = {'accuracy', 'neg_log_loss', 'roc_auc'} for evaluation metric - althought they are many
scores_accuracy = cross_val_score(logreg, X, y, cv=10, scoring='accuracy')
scores_log_loss = cross_val_score(logreg, X, y, cv=10, scoring='neg_log_loss')
scores_auc = cross_val_score(logreg, X, y, cv=10, scoring='roc_auc')
print('K-fold cross-validation results:')
print(logreg.__class__.__name__+" average accuracy is %2.3f" % scores_accuracy.mean())
print(logreg.__class__.__name__+" average log_loss is %2.3f" % -scores_log_loss.mean())
print(logreg.__class__.__name__+" average auc is %2.3f" % scores_auc.mean())

K-fold cross-validation results:
LogisticRegression average accuracy is 0.687
LogisticRegression average log_loss is 0.611
LogisticRegression average auc is 0.586


In [0]:
# load the model from disk
# loaded_model = joblib.load(filename)
# result = loaded_model.score(X_test, Y_test)

In [0]:
final_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/test_final.csv", sep = ',')

In [0]:
final_test.replace([np.inf, -np.inf], np.nan, inplace=True)
final_test.fillna(0, inplace=True)

In [0]:
logreg.max_iter = 10000
logreg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
X.head(5)

,prop_brand_bool,prop_log_historical_price,promotion_flag,srch_length_of_stay,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,histpricediff,histstardiff,pricediff,stardiff,reviewscorediff,locationscore1diff,locationscore2diff,pricechange,hotel_count,popularity_score,is219,usd_diff,prop_starrating_monotonic,room_count_booking_window,people_stay_count,month
0,0,4.14,0,1,1,0.0,386.60,1,0.0,0.0,0.0,66.87,1.45,-78.348929,-1.071429,-3.482143,-0.689643,-0.045493,0.555576,104,104.000000,0,66.87,1.31206,0,4,4
1,1,4.44,0,1,1,0.0,386.60,1,0.0,0.0,0.0,51.35,1.45,-62.828929,-1.071429,-0.482143,-0.099643,-0.024893,0.625685,584,496.400000,0,51.35,1.31206,0,4,4
2,0,5.39,0,2,1,0.0,236.91,1,0.0,0.0,0.0,-66.76,3.45,-46.562500,-3.093750,0.593750,-0.894063,-0.081466,0.000000,168,56.000000,1,66.76,3.31206,46,1,12
3,1,5.92,0,2,1,0.0,238.30,1,0.0,0.0,0.0,-176.76,-0.55,63.437500,0.906250,0.593750,0.585937,-0.022066,0.100703,507,295.750000,1,176.76,0.68794,46,1,12
4,1,5.25,0,2,1,0.0,236.14,1,0.0,0.0,0.0,23.24,1.45,-136.562500,-1.093750,-0.906250,-0.894063,-0.132866,0.040323,358,208.833333,1,23.24,1.31206,46,1,12


In [0]:
import joblib
filename = '/content/drive/My Drive/Colab Notebooks/saved_model/logreg_model.sav'
joblib.dump(logreg, filename)

['/content/drive/My Drive/Colab Notebooks/saved_model/logreg_model.sav']

In [0]:
final_test['target'] = logreg.predict(final_test[selected_features])

In [0]:
final_test.sort_values(["srch_id","target"],ascending=[True,False], inplace=True)

In [0]:
final_test[final_test.srch_id == 28].head(5)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,promotion_flag,srch_destination_id,srch_length_of_stay,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,popularity_score,histpricediff,histstardiff,pricediff,stardiff,reviewscorediff,locationscore1diff,locationscore2diff,pricechange,hotel_count,is219,usd_diff,prop_starrating_monotonic,room_count_booking_window,people_stay_count,month,target
433,28,5,219,3.43,151.39,219,27788,4.0,0,2.20,0.3131,3.82,1,3422,4,1,0.0,222.80,0,0.0,0.0,0.0,255.200000,120.39,-0.57,-21.083333,1.583333,0.625,0.475833,0.191992,0.107143,273,1,120.39,0.817458,50,1,4,2
432,28,5,219,3.43,151.39,219,9696,4.0,0,2.30,0.1795,4.05,0,3422,4,1,0.0,222.60,0,0.0,0.0,0.0,184.571429,96.39,-0.57,2.916667,1.583333,0.625,0.575833,0.058392,0.100000,222,1,96.39,0.817458,50,1,4,2
439,28,5,219,3.43,151.39,219,118188,3.5,0,2.20,0.3028,3.52,1,3422,4,1,0.0,222.33,0,0.0,0.0,0.0,292.923077,126.39,0.43,-27.083333,0.583333,0.125,0.475833,0.181692,0.315789,282,1,126.39,0.182542,50,1,4,2
440,28,5,219,3.43,151.39,219,119653,4.0,0,2.20,0.2134,4.29,0,3422,4,1,0.0,223.08,0,0.0,0.0,0.0,225.135135,101.39,-0.57,-2.083333,1.583333,0.625,0.475833,0.092292,0.315789,291,1,101.39,0.817458,50,1,4,2
436,28,5,219,3.43,151.39,219,96426,3.5,1,1.61,0.2168,3.89,0,3422,4,1,0.0,221.97,0,0.0,0.0,0.0,230.547945,106.39,-0.57,-7.083333,1.583333,0.125,-0.114167,0.095692,1.142857,276,1,106.39,0.817458,50,1,4,0


In [0]:
final_test.head(20)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,promotion_flag,srch_destination_id,srch_length_of_stay,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,popularity_score,histpricediff,histstardiff,pricediff,stardiff,reviewscorediff,locationscore1diff,locationscore2diff,pricechange,hotel_count,is219,usd_diff,prop_starrating_monotonic,room_count_booking_window,people_stay_count,month,target
5,1,24,216,3.43,151.39,219,28181,4.5,1,2.30,0.1826,4.68,0,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,335.172414,67.24,0.43,-9.646552,0.310345,0.482759,-0.36069,0.057217,0.001786,381,0,67.24,0.182542,10,2,2,2
6,1,24,216,3.43,151.39,219,34263,4.5,1,3.09,0.1300,4.63,0,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,187.000000,72.39,0.43,-14.796552,0.310345,0.482759,0.42931,0.004617,0.082192,202,0,72.39,0.182542,10,2,2,2
9,1,24,216,3.43,151.39,219,54937,4.0,1,2.08,0.1649,4.75,1,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,341.739130,68.09,0.43,-10.496552,0.310345,-0.017241,-0.58069,0.039517,0.041250,403,0,68.09,0.182542,10,2,2,2
12,1,24,216,3.43,151.39,219,61934,4.5,1,2.89,0.2425,4.69,1,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,277.692308,62.51,0.43,-4.916552,0.310345,0.482759,0.22931,0.117117,0.125063,377,0,62.51,0.182542,10,2,2,2
16,1,24,216,3.43,151.39,219,74045,4.0,1,2.77,0.2003,4.56,0,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,215.000000,61.40,0.43,-3.806552,0.310345,-0.017241,0.10931,0.074917,0.184079,213,0,61.40,0.182542,10,2,2,2
23,1,24,216,3.43,151.39,219,99484,4.0,1,2.40,0.2182,4.54,1,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,288.190476,82.39,0.43,-24.796552,0.310345,-0.017241,-0.26069,0.092817,0.000000,368,0,82.39,0.182542,10,2,2,2
7,1,24,216,3.43,151.39,219,37567,4.5,0,2.83,0.0138,4.21,1,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,159.000000,98.99,1.43,-41.396552,-0.689655,0.482759,0.16931,-0.111583,0.007692,175,0,98.99,1.182542,10,2,2,2
8,1,24,216,3.43,151.39,219,50162,3.5,1,2.20,0.0808,4.36,1,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,243.333333,101.40,1.43,-43.806552,-0.689655,-0.517241,-0.46069,-0.044583,-0.000200,293,0,101.40,1.182542,10,2,2,2
0,1,24,216,3.43,151.39,219,3180,4.5,1,2.94,0.0691,5.03,0,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,186.000000,32.39,0.43,25.203448,0.310345,0.482759,0.27931,-0.056283,-0.032520,185,0,32.39,0.182542,10,2,2,2
4,1,24,216,3.43,151.39,219,24194,4.5,1,2.94,0.2090,4.72,0,19222,1,0,0.0,387.85,0,0.0,0.0,0.0,153.750000,72.39,0.43,-14.796552,0.310345,0.482759,0.27931,0.083617,-0.112360,214,0,72.39,0.182542,10,2,2,2


In [0]:
submission = final_test[['srch_id','prop_id']]

In [0]:
submission.head(5)

,srch_id,prop_id
5,1,28181
6,1,34263
9,1,54937
12,1,61934
16,1,74045


In [0]:
submission.to_csv("/content/drive/My Drive/Colab Notebooks/submission_logreg.csv",index=False)